In [ ]:
%load_ext autoreload
%autoreload 2

### Generate a random Hamiltonian instance

* We can generate various classes
* Instances can be generated on the fly or read from file
* We can also solve the Hamiltonian classicaly, to get the ground state energy and the highest energy.



In [ ]:
from quapopt.data_analysis.data_handling import (COEFFICIENTS_TYPE,
                                                 COEFFICIENTS_DISTRIBUTION,
                                                 CoefficientsDistributionSpecifier,
                                                 HAMILTONIAN_MODELS)
from quapopt.hamiltonians.generators import build_hamiltonian_generator

hamiltonian_model = HAMILTONIAN_MODELS.SherringtonKirkpatrick

coefficients_type = COEFFICIENTS_TYPE.DISCRETE
coefficients_distribution = COEFFICIENTS_DISTRIBUTION.Uniform
coefficients_distribution_properties = {'low': -1, 'high': 1, 'step': 1}

coefficients_distribution_specifier = CoefficientsDistributionSpecifier(CoefficientsType=coefficients_type,
                                                                        CoefficientsDistributionName=coefficients_distribution,
                                                                        CoefficientsDistributionProperties=coefficients_distribution_properties)

arguments_dict_builder = {}

localities = None
if hamiltonian_model in [HAMILTONIAN_MODELS.SherringtonKirkpatrick, ]:
    localities = (1, 2)
elif hamiltonian_model in [HAMILTONIAN_MODELS.RegularGraph]:
    localities = (2,)
elif hamiltonian_model in [HAMILTONIAN_MODELS.ErdosRenyi]:
    localities = (1, 2)
elif hamiltonian_model in [HAMILTONIAN_MODELS.MaxCut]:
    pass
elif hamiltonian_model in [HAMILTONIAN_MODELS.MAX2SAT]:
    pass
elif hamiltonian_model in [HAMILTONIAN_MODELS.Wishart]:
    pass

generator_cost_hamiltonian = build_hamiltonian_generator(hamiltonian_model=hamiltonian_model,
                                                         localities=localities,
                                                         coefficients_distribution_specifier=coefficients_distribution_specifier)

In [ ]:
solve_hamiltonian = True
number_of_qubits = 25
arguments_instance = {'number_of_qubits': number_of_qubits,
                      'seed': 0}
if hamiltonian_model in [HAMILTONIAN_MODELS.SherringtonKirkpatrick, ]:
    pass
elif hamiltonian_model in [HAMILTONIAN_MODELS.RegularGraph]:
    graph_degree = 3
    arguments_instance['graph_degree'] = graph_degree
elif hamiltonian_model in [HAMILTONIAN_MODELS.ErdosRenyi]:
    average_degree = number_of_qubits // 2
    edge_probability = average_degree / (number_of_qubits - 1)
    arguments_instance['p_or_M'] = edge_probability
elif hamiltonian_model in [HAMILTONIAN_MODELS.MaxCut]:
    average_degree = number_of_qubits // 2
    edge_probability = average_degree / (number_of_qubits - 1)
    arguments_instance['p_or_M'] = edge_probability
elif hamiltonian_model in [HAMILTONIAN_MODELS.MAX2SAT]:
    clause_density = 1.1
    arguments_instance['clause_density'] = clause_density

elif hamiltonian_model in [HAMILTONIAN_MODELS.WishartPlantedEnsemble]:
    pass

cost_hamiltonian = generator_cost_hamiltonian.generate_instance(**arguments_instance)


if solve_hamiltonian:
    print("SOLVING HAMILTONIAN")
    #This will write known solutions if they already exist (if not, it doesn't do anything)
    cost_hamiltonian.write_solutions_to_file()
    #This solving is required to get lowest energy state and highest energy state
    cost_hamiltonian.solve_hamiltonian(both_directions=True,
                                       solver_kwargs={'solver_name': "BURER2002",
                                                      'solver_timeout': 1})

ground_state_energy = cost_hamiltonian.ground_state_energy
highest_energy = cost_hamiltonian.highest_energy

print("Ground state energy:", ground_state_energy)
print("Highest energy:", highest_energy)